In [24]:
import numpy as np
import pandas as pd
import random

In [52]:
class Network(object):
    """
    'object' is an identifier
    """
    def __init__(self,sizes):
        """
        sizes refers to the number of layers and the number of neurons in each layer.
        For example,
        Network([2,3,1]) constructs an object NN with three layers and respectively 
        2, 3, and 1 neurons in each layer.
        """
        self.num_layers = len(sizes)
        self.sizes = sizes
        self.biases = [np.random.randn(y,1) for y in sizes[1:]]
        self.weights = [np.random.randn(y,x)
                       for x,y in zip(sizes[:-1],sizes[1:])]
    def feedforward(self,a):
        """
        This implements the feed forward algorithm to build the activations vector 
        sequentially for each layer. It receives an input numpy ndarray and returns 
        the output for the network
        """
        for b,w in zip(self.biases,self.weights):
            a = sigmoid(np.dot(w,a) + b)
        return a
    def SGD(self, training_data,epochs,mini_batch_size, eta, test_data = None):
        """
        training_data: tuple(x,y) where x is the input and y is the desired output
        epochs: number of complete iterations over the training data set
        mini_batch_size: number of training data samples in the batch for SGD
        eta:  the greek symbol. Learning rate
        test_data: self-explanatory
        """
        if test_data:
            n_test = len(test_data)
        n = len(training_data)
        for j in range(epochs):
            random.shuffle(training_data)
            mini_batches = [
                training_data[k:k+mini_batch_size]
                for k in range(0,n,mini_batch_size)                
            ]
            for mini_batch in mini_batches:
                self.update_mini_batch(mini_batch,eta)
            if test_data:
                print("Epoch {0}: {1}/{2}".format(
                j, self.evaluate(test_data),n_test))
            else:
                print("Epoch {0} complete".format(j))
                
    def update_mini_batch(self, mini_batch, eta):
        """
        This is where the actual learning occurs. The minibatches are updated using
        backpropagation (an application of Gradient Descent).
        mini_batch: sampled training_data
        eta: learning rate defined in the SGD method
        addendum--nabla is the greek symbol for the gradient of associated variable
        """
        nabla_b = [np.zeros(b.shape) for b in self.biases] #required change in biases(gradient)
        nabla_w = [np.zeros(w.shape) for w in self.weights]#required change in weights(gradient)
        for x,y in mini_batch:
            delta_nabla_b, delta_nabla_w = self.backprop(x,y) #actual learning occurs here
            nabla_b = [nb+dnb for nb,dnb in zip(nabla_b,delta_nabla_b)]
            nabla_w = [nw+dnw for nw,dnw in zip(nabla_w,delta_nabla_w)]
        self.weights = [w - (eta/len(mini_batch))*nw
                       for w,nw in zip(self.weights,nabla_w)]
        self.biases = [b - (eta/len(mini_batch))*nb
                      for b,nb in zip(self.biases,nabla_b)]
        
    def backprop(self, x, y):
        """Return a tuple ``(nabla_b, nabla_w)`` representing the
        gradient for the cost function C_x.  ``nabla_b`` and
        ``nabla_w`` are layer-by-layer lists of numpy arrays, similar
        to ``self.biases`` and ``self.weights``."""
        nabla_b = [np.zeros(b.shape) for b in self.biases]
        nabla_w = [np.zeros(w.shape) for w in self.weights]
        # feedforward
        activation = x
        activations = [x] # list to store all the activations, layer by layer
        zs = [] # list to store all the z vectors, layer by layer
        for b, w in zip(self.biases, self.weights):
            z = np.dot(w, activation)+b
            zs.append(z)
            activation = sigmoid(z)
            activations.append(activation)
        # backward pass
        delta = self.cost_derivative(activations[-1], y) * \
            sigmoid_prime(zs[-1])
        nabla_b[-1] = delta
        nabla_w[-1] = np.dot(delta, activations[-2].transpose())
        # Note that the variable l in the loop below is used a little
        # differently to the notation in Chapter 2 of the book.  Here,
        # l = 1 means the last layer of neurons, l = 2 is the
        # second-last layer, and so on.  It's a renumbering of the
        # scheme in the book, used here to take advantage of the fact
        # that Python can use negative indices in lists.
        for l in range(2, self.num_layers):
            z = zs[-l]
            sp = sigmoid_prime(z)
            delta = np.dot(self.weights[-l+1].transpose(), delta) * sp
            nabla_b[-l] = delta
            nabla_w[-l] = np.dot(delta, activations[-l-1].transpose())
        return (nabla_b, nabla_w)

    def evaluate(self, test_data):
        """Return the number of test inputs for which the neural
        network outputs the correct result. Note that the neural
        network's output is assumed to be the index of whichever
        neuron in the final layer has the highest activation."""
        test_results = [(np.argmax(self.feedforward(x)), y)
                        for (x, y) in test_data]
        return sum(int(x == y) for (x, y) in test_results)

    def cost_derivative(self, output_activations, y):
        """Return the vector of partial derivatives \partial C_x /
        \partial a for the output activations."""
        return (output_activations-y)

In [53]:
def sigmoid(z):
    """This function is the activation function. For a numpy vector, the function is vectorized"""
    return 1.0/(1.0+np.exp(-z))

In [54]:
def sigmoid_prime(z):
    """Derivative of the sigmoid function."""
    return sigmoid(z)*(1-sigmoid(z))

In [55]:
"""
mnist_loader
~~~~~~~~~~~~

A library to load the MNIST image data.  For details of the data
structures that are returned, see the doc strings for ``load_data``
and ``load_data_wrapper``.  In practice, ``load_data_wrapper`` is the
function usually called by our neural network code.
"""

#### Libraries
# Standard library
import pickle
import gzip

# Third-party libraries
import numpy as np

def load_data():
    """Return the MNIST data as a tuple containing the training data,
    the validation data, and the test data.

    The ``training_data`` is returned as a tuple with two entries.
    The first entry contains the actual training images.  This is a
    numpy ndarray with 50,000 entries.  Each entry is, in turn, a
    numpy ndarray with 784 values, representing the 28 * 28 = 784
    pixels in a single MNIST image.

    The second entry in the ``training_data`` tuple is a numpy ndarray
    containing 50,000 entries.  Those entries are just the digit
    values (0...9) for the corresponding images contained in the first
    entry of the tuple.

    The ``validation_data`` and ``test_data`` are similar, except
    each contains only 10,000 images.

    This is a nice data format, but for use in neural networks it's
    helpful to modify the format of the ``training_data`` a little.
    That's done in the wrapper function ``load_data_wrapper()``, see
    below.
    """
    f = gzip.open('data/yetanothermnist.pkl.gz', 'rb')
    training_data, validation_data, test_data = pickle.load(f,encoding='latin1')
    f.close()
    return (training_data, validation_data, test_data)

def load_data_wrapper():
    """Return a tuple containing ``(training_data, validation_data,
    test_data)``. Based on ``load_data``, but the format is more
    convenient for use in our implementation of neural networks.

    In particular, ``training_data`` is a list containing 50,000
    2-tuples ``(x, y)``.  ``x`` is a 784-dimensional numpy.ndarray
    containing the input image.  ``y`` is a 10-dimensional
    numpy.ndarray representing the unit vector corresponding to the
    correct digit for ``x``.

    ``validation_data`` and ``test_data`` are lists containing 10,000
    2-tuples ``(x, y)``.  In each case, ``x`` is a 784-dimensional
    numpy.ndarry containing the input image, and ``y`` is the
    corresponding classification, i.e., the digit values (integers)
    corresponding to ``x``.

    Obviously, this means we're using slightly different formats for
    the training data and the validation / test data.  These formats
    turn out to be the most convenient for use in our neural network
    code."""
    tr_d, va_d, te_d = load_data()
    training_inputs = [np.reshape(x, (784, 1)) for x in tr_d[0]]
    training_results = [vectorized_result(y) for y in tr_d[1]]
    training_data = list(zip(training_inputs, training_results))
    validation_inputs = [np.reshape(x, (784, 1)) for x in va_d[0]]
    validation_data = list(zip(validation_inputs, va_d[1]))
    test_inputs = [np.reshape(x, (784, 1)) for x in te_d[0]]
    test_data = list(zip(test_inputs, te_d[1]))
    return (training_data, validation_data, test_data)

def vectorized_result(j):
    """Return a 10-dimensional unit vector with a 1.0 in the jth
    position and zeroes elsewhere.  This is used to convert a digit
    (0...9) into a corresponding desired output from the neural
    network."""
    e = np.zeros((10, 1))
    e[j] = 1.0
    return e



In [56]:
training_data, validation_data, test_data = load_data_wrapper()

In [57]:
net = Network([784, 30, 10])

In [58]:
net.SGD(training_data, 30, 10, 3.0, test_data=test_data)

Epoch 0: 9122/10000
Epoch 1: 9243/10000
Epoch 2: 9333/10000
Epoch 3: 9369/10000
Epoch 4: 9348/10000
Epoch 5: 9383/10000
Epoch 6: 9403/10000
Epoch 7: 9464/10000
Epoch 8: 9452/10000
Epoch 9: 9462/10000
Epoch 10: 9448/10000
Epoch 11: 9456/10000
Epoch 12: 9471/10000
Epoch 13: 9459/10000
Epoch 14: 9465/10000
Epoch 15: 9468/10000
Epoch 16: 9482/10000
Epoch 17: 9460/10000
Epoch 18: 9460/10000
Epoch 19: 9464/10000
Epoch 20: 9473/10000
Epoch 21: 9484/10000
Epoch 22: 9483/10000
Epoch 23: 9459/10000
Epoch 24: 9488/10000
Epoch 25: 9478/10000
Epoch 26: 9482/10000
Epoch 27: 9510/10000
Epoch 28: 9481/10000
Epoch 29: 9510/10000


In [102]:
for x in net.weights[0]:
    print(x)

[ -1.08675586e+00   8.43480741e-02   8.80182108e-01   1.46905466e-01
  -4.37141495e-01   3.47188033e-01   1.34355790e+00   1.19065126e+00
   3.65124034e-01   3.39191078e-01   3.78283296e-01   2.09662788e+00
  -7.15012689e-01  -1.91046904e+00  -1.67829765e+00  -1.17665561e+00
   1.03670434e+00   1.89157303e-01   1.08250976e+00   1.24303264e+00
   1.16854029e+00   2.31864682e+00   1.44088435e+00   7.66989104e-01
  -1.88339725e+00   1.05146772e+00  -5.53953481e-01   3.19873559e-02
   2.54704558e-01   6.41297772e-01   1.20047856e+00   7.26684768e-01
  -5.75870854e-01   4.94959156e-01  -8.30131685e-01   5.05156360e-01
  -1.01187795e+00   2.62758248e-01  -3.99563632e-01   1.83356284e+00
   4.81444137e-01   2.25692805e+00   1.01019511e+00  -1.05835029e+00
   2.28299975e+00  -1.02172878e+00   6.62749310e-01   3.96184709e-01
  -1.20926714e+00  -9.73731260e-01   2.35367006e-02   1.39446059e+00
   5.51766419e-01  -1.24553025e+00   1.28969368e+00  -8.08580766e-01
  -2.87121363e-01  -9.33688779e-01

[-0.8977092  -0.85351153 -0.75257601 -0.13804205  0.10327252 -1.30336113
 -0.35531607 -1.73786705  1.5309494  -0.51612829  1.55429544 -1.11338724
  1.05461617 -0.70939154  1.77117345  1.2931971   1.10294715  0.3922867
 -0.31208387  0.87915576 -0.46654602  0.29928465 -0.83921928  1.04344847
 -0.15778318 -1.04966897 -1.52505067 -0.26905649 -0.37529004  0.76061844
 -1.82218496  0.17609333  1.9150505   0.2306366   2.35907602  0.25917679
  0.42352055  0.57074086 -0.44038121 -0.67574211 -0.47758798  0.66901211
 -1.90474541  0.67662897  0.9080246  -1.0931402   1.32042656 -0.7701089
 -0.54131815  1.49829143 -0.09563926 -0.59073944  0.54061919  0.51210827
  0.36351578  0.86412784  0.52687552 -0.96162395 -0.15248944 -2.21255091
 -1.67863412  0.96669299 -1.30945183 -2.47604794  0.02749724 -0.84363585
  1.56749017 -1.67576357 -1.38261839 -0.31817361 -0.46789123  0.29500746
 -0.52144674 -0.04207457 -1.28180944 -0.74788747  0.79462736  1.19247589
 -0.11728872 -0.01102816  3.56106884  0.20381375  0.4

[  2.38458291e-01   3.46431836e-01  -8.52405326e-01   5.03047630e-01
  -4.93939063e-02   1.84546881e+00  -3.03995385e-01   8.15629050e-01
  -7.30496972e-01  -1.05644710e-01   1.13759544e-01  -4.40438678e-01
   5.53969984e-01  -2.80647860e-01  -8.43539749e-01  -7.14865272e-02
   1.30074602e+00   4.49303044e-02   2.15086846e+00  -2.49202878e-01
  -1.60203798e-01   9.12041242e-01  -4.02751077e-01  -3.01331069e-02
   4.42763667e-01   2.80460048e-01   9.10308763e-01  -1.82493538e-01
  -3.63378418e-02   6.22428903e-01  -2.61019701e-01   1.09975295e+00
  -9.63341189e-01  -7.67641211e-01  -2.25709731e+00  -1.04236579e+00
  -1.68627527e+00  -3.18263456e-01  -7.25768735e-01   1.07657599e+00
   3.17501603e-01  -1.55304614e-01  -3.40963123e-01  -9.59305423e-01
  -7.71852303e-02  -1.30578106e-01   1.36674419e+00   2.60888419e-02
  -9.28063083e-01   1.92830576e-01   1.26054751e+00  -5.93759285e-01
  -8.71676671e-01  -1.29357223e+00   1.00805128e+00   1.27355049e+00
   8.45828851e-01  -6.24695715e-01

[ -9.08121377e-01   1.20627011e-01   8.80887618e-01   1.58887022e+00
   1.98899160e-01   1.74097588e-01   1.09065708e-01   9.03851609e-01
  -1.25255647e+00  -1.27111835e+00   2.08643825e+00   3.14230003e-02
  -1.03629234e+00  -3.24983586e-01   6.79921247e-01   3.52968137e-01
  -1.74557284e-01   1.52965080e+00  -7.33101980e-01   2.07482492e+00
   7.51256918e-02   1.52379881e+00   1.09062477e+00  -8.41855821e-01
   8.16440114e-01  -7.42558700e-02   8.59124246e-01   5.14898695e-01
   1.36554648e+00  -1.61698217e+00   2.14690748e-01  -1.37016686e+00
   2.77337023e-01  -1.58710226e-01   5.57292951e-01  -3.49930319e-01
  -1.21713530e+00  -1.30497461e-01   1.03399740e+00  -6.31827578e-01
  -1.62871496e+00   3.37324884e-02   1.98257935e-01   6.60712727e-01
  -2.94682045e-01  -1.35887937e+00  -1.43741246e+00   6.94305138e-01
  -2.08482344e-01  -6.99707473e-01  -3.40502726e-01   2.92664658e-01
  -2.44391092e+00   1.60911050e-01   1.38121566e+00   8.56174829e-01
  -7.07986983e-01   1.62827330e+00

[ -1.46351746e+00   9.67447744e-01  -1.09117048e+00   7.53147968e-01
  -3.75721499e-01   3.01214121e-02  -3.83133154e-01  -1.41771900e+00
   8.78336906e-01   1.18802472e+00   1.06866570e+00  -1.53470492e+00
   6.67547140e-01  -1.06849579e+00  -1.10936413e+00   1.07907491e+00
  -5.79511395e-01   1.61940026e+00  -1.03963728e+00   6.61177527e-02
  -9.90464359e-01   8.31114608e-01   5.21390556e-01  -1.42686478e+00
  -5.22731562e-01  -4.83670527e-01  -1.15361908e-01  -7.69322465e-01
  -7.78137707e-01  -3.16845123e-01   1.29093896e-02  -8.98767156e-01
  -4.88071427e-01   6.73871167e-01  -1.20863624e+00  -1.17046006e+00
  -1.44393886e+00  -1.11604034e-01   1.95342301e+00  -1.12308825e+00
   4.03800249e-02  -1.48607086e+00  -1.80383835e-01   5.58269713e-01
  -5.77328663e-01  -8.47599976e-01   1.42524949e+00   2.81477935e-01
   5.66621673e-01  -3.34699609e-01  -9.26543647e-01   1.48814894e+00
  -2.26063079e-01  -1.68748236e-01  -1.12675620e+00  -1.12502159e+00
  -1.10479319e+00  -9.41852737e-01

...

[  1.87068842e+00  -1.16962141e+00   1.32407552e+00  -1.33482788e+00
   7.72350828e-02   3.97805764e-01   1.47359042e+00   5.11533043e-01
   5.57486392e-01   1.41014054e+00  -6.61864383e-01  -1.28869678e-01
  -7.09234336e-01  -1.08951770e-01   7.49604241e-01  -1.34503046e+00
   2.26964961e+00  -1.43250540e+00  -1.75830888e+00   2.09999193e+00
  -3.59304176e-01   2.48434125e-01  -2.14887370e+00  -1.37287560e+00
  -1.48196628e+00   1.11785118e-01   2.39880347e-01   3.15228424e-01
  -2.53874777e+00   6.11548673e-01   1.30642431e+00   9.06411810e-01
  -1.40844303e+00   1.31344065e-01   1.39840020e-01   1.22365429e+00
  -8.56273375e-01  -1.32168027e-01   1.95025592e+00  -2.96267413e-01
  -5.14124370e-01   1.50735962e-01  -1.15228321e+00   1.47950785e+00
   6.89530458e-01  -7.94721110e-01  -1.07263074e+00   9.44720389e-01
  -3.06635975e-01  -4.72977672e-01  -4.94707722e-01   1.85419190e+00
  -8.35622081e-01  -2.34076263e-01  -2.41687273e+00  -6.01748780e-01
   1.58272274e+00   2.26584842e+00

[ -8.96618043e-01  -1.51419981e+00   2.35225605e+00   9.21690144e-01
   1.01438375e+00   7.42757163e-02   1.57669928e-01   8.01830451e-01
  -2.98253853e+00   1.89390685e-01  -1.60279470e+00  -1.01655092e+00
   7.26644184e-01   1.61839851e-01  -1.68035312e+00   4.30169280e-01
   2.46308759e-02  -6.15453048e-01  -3.84724631e-01   1.56927053e+00
  -3.15486361e-01  -2.31835656e-01  -9.44687331e-03   1.34582447e+00
  -6.67577905e-01  -2.94739604e-01  -3.69826116e-02   2.19842723e-01
  -1.55683048e+00   1.38313109e+00   7.08198395e-01   1.31368192e+00
  -9.24434240e-02  -8.70168104e-01  -1.06379386e+00   5.18693186e-01
  -1.38389062e+00   7.24263689e-01   9.52982983e-02   9.69858329e-01
   9.53479196e-01  -6.83082033e-01   6.00511851e-02  -5.61657546e-01
  -9.92941483e-01   1.90962889e+00   4.09996896e-01   3.87411994e-01
   8.80725773e-01   8.75766836e-01  -7.99994501e-04  -1.50234744e+00
   5.67826276e-01  -1.26076315e-01  -1.71387235e+00  -1.14772118e+00
  -2.21249222e-01  -3.49861489e-01

[ -2.32715079e+00   1.36641702e+00   4.75691629e-01   2.44883710e-01
   4.38546953e-01   1.72398909e-01   1.44889101e+00  -1.13491834e+00
  -2.58382249e-01  -5.41717320e-02   2.38689063e-01  -1.40249310e+00
  -3.19821631e-01  -2.84895244e+00  -7.24469018e-01   1.08165560e+00
   5.48796573e-01   5.40781389e-01   4.45937953e-01  -6.94375651e-01
   2.18868897e-01   1.55972676e+00  -1.23244318e+00  -3.28584232e-01
   5.63811499e-01   5.63488125e-01  -1.20031592e+00  -1.11726691e+00
  -8.13600664e-01  -1.37652022e+00   7.48276920e-01   7.15654544e-01
   5.73338076e-01  -3.23005026e-01  -1.93333570e-01   3.62159851e-01
  -1.12280191e+00   1.88598063e-01   2.52210492e-01   1.49425513e+00
  -1.59938478e+00   1.64617168e+00  -1.78857712e-01  -2.25822743e-01
  -1.09092967e+00  -1.12296817e+00   1.43017156e+00  -2.03860384e+00
  -9.63824635e-01   3.03302571e-01  -2.46670835e-01  -5.84370701e-01
   4.93972634e-01  -8.56856286e-01   6.61192758e-01   1.61686920e-01
   3.86954165e-01  -2.35780864e-01

[  4.12132050e-01   1.13664450e-01  -8.85249810e-01   2.39499683e-01
   8.93765675e-01  -6.02322214e-01  -1.07393409e+00   2.06901476e-01
  -1.07918394e+00   1.32258656e+00   5.59758109e-01   2.53596176e+00
  -1.44713037e+00   1.86098425e+00   4.08751344e-02  -3.23696219e-01
  -1.31561831e+00  -6.10309467e-01  -9.48107319e-01  -8.69289923e-01
   4.21995310e-01  -2.31957542e-01  -4.80520457e-01  -1.01241253e+00
  -1.22227094e+00  -5.76358272e-01  -6.95886855e-01  -5.82642646e-01
   9.44988904e-01  -6.63657422e-02  -2.30014762e-01   1.70804372e-01
   5.30147602e-01  -8.57162912e-01   1.19293863e-02   7.39150494e-01
   1.69371861e-01  -7.59257131e-01   1.17816237e+00  -8.38034476e-01
   1.97938540e-01   7.93104121e-02   8.04589813e-03   9.79841059e-01
  -1.73975031e+00   1.94993726e-01  -2.51684562e-01   4.07424737e-01
  -1.86375023e-01   1.16238526e+00  -3.07810272e-01   1.54240723e+00
  -1.20607197e-01  -6.80355733e-01  -1.87058618e+00   1.44011565e-01
   2.18765080e+00   1.29841299e+00

[  3.28291873e-01  -3.82993322e-01   2.97488854e+00  -2.31754084e+00
  -2.01145618e+00   6.32791354e-01   2.61558171e+00  -3.01725578e-01
   1.96338028e+00  -2.68597207e+00  -1.73252173e+00   7.09220457e-01
   1.45275378e+00  -6.21251321e-01  -1.13885036e+00   1.29144566e+00
   1.87706809e+00   4.37693867e-01   1.82876226e-01  -1.16502435e+00
   9.53189393e-01   9.09891451e-01  -1.69895987e-01   9.33422348e-01
  -6.33102571e-01  -7.41346881e-01   4.92377611e-02   6.37510396e-01
  -5.43926545e-01  -3.81510889e-02  -1.67661072e+00   1.79423732e+00
  -8.92593960e-01   5.67263722e-01  -1.74049419e+00   2.53499237e-01
   1.36548773e-01  -8.74394277e-01  -3.80496754e-01  -1.66603773e+00
  -1.49951298e-01  -5.59849035e-01   1.02400271e+00   7.24470182e-01
  -1.87858403e-01   9.06624383e-01   2.74531665e+00  -1.62801261e+00
  -1.05881213e+00   2.90423496e-01   1.22034477e+00   1.52637149e+00
   6.59507940e-02  -1.32036333e+00   3.03596918e-01  -4.85925877e-01
   5.32728285e-01   5.02907753e-01

In [60]:
net.biases

[array([[ 2.128638  ],
        [ 1.45137528],
        [ 1.9858791 ],
        [ 0.48412635],
        [-0.36693302],
        [ 0.2536644 ],
        [ 1.56591106],
        [ 0.8168221 ],
        [ 0.95525105],
        [-3.67584526],
        [ 3.86726499],
        [-3.18136134],
        [-1.67168328],
        [-0.26771153],
        [-4.16617381],
        [-1.38694641],
        [ 1.80797592],
        [-2.63806319],
        [-1.54124148],
        [-4.5603659 ],
        [ 2.48615489],
        [-0.05244362],
        [-1.32417332],
        [ 1.00074697],
        [ 1.90375822],
        [-0.6643711 ],
        [-3.69286707],
        [ 0.63011889],
        [-3.95152747],
        [-2.24033226]]), array([[-3.90598162],
        [-2.03999821],
        [-3.59747705],
        [-4.27942903],
        [-4.96335248],
        [-0.53053458],
        [-4.57238514],
        [-3.40355255],
        [-6.09197484],
        [-5.22684303]])]